<a href="https://colab.research.google.com/github/vishal-burman/PyTorch-Architectures/blob/master/research/modeling_NiN/test_sample_NiN(BatchNorm_after_activation).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/vishal-burman/PyTorch-Architectures.git

Cloning into 'PyTorch-Architectures'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (139/139), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 1442 (delta 80), reused 74 (delta 33), pack-reused 1303
Receiving objects: 100% (1442/1442), 8.57 MiB | 21.99 MiB/s, done.
Resolving deltas: 100% (848/848), done.


In [2]:
%cd PyTorch-Architectures/research/modeling_NiN/

/content/PyTorch-Architectures/research/modeling_NiN


In [3]:
import time

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader, Subset

from torchvision import datasets, transforms

from NiN_bn_after_activation import NiN

if torch.cuda.is_available():
  torch.backends.cudnn.deterministic = True

In [4]:
#############
# Settings
#############

RANDOM_SEED = 1
LEARNING_RATE = 0.0005
BATCH_SIZE = 256
EPOCHS = 10

# Architecture
NUM_CLASSES = 10

# Other
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
GRAYSCALE = False

In [5]:
########################
# CIFAR-10
########################

train_indices = torch.arange(0, 49000)
valid_indices = torch.arange(49000, 50000)

train_and_valid = datasets.CIFAR10(root='data',
                                   train=True,
                                   transform=transforms.ToTensor(),
                                   download=True)

train_dataset = Subset(train_and_valid, train_indices)
valid_dataset = Subset(train_and_valid, valid_indices)

test_dataset = datasets.CIFAR10(root='data',
                                train=False, 
                                transform=transforms.ToTensor())

########################
# DataLoaders
########################

train_loader = DataLoader(dataset=train_dataset,
                          batch_size=BATCH_SIZE,
                          num_workers=4,
                          shuffle=True)
valid_loader = DataLoader(dataset=valid_dataset,
                          batch_size=BATCH_SIZE,
                          num_workers=4,
                          shuffle=False)
test_loader = DataLoader(dataset=test_dataset,
                         batch_size=BATCH_SIZE,
                         num_workers=4,
                         shuffle=False)

############################
# Check dataset
############################
for images, labels in train_loader:
  print('Images Dimensions: ', images.shape)
  print('Labels Dimensions: ', labels.shape, "\n")
  break

for images, labels in valid_loader:
  print('Images Dimensions: ', images.shape)
  print('Labels Dimensions: ', labels.shape, "\n")
  break

for images, labels in test_loader:
  print('Images Dimensions: ', images.shape)
  print('Labels Dimensions: ', labels.shape, "\n")
  break

print('Length of train_loader: ', len(train_loader))
print('Length of valid_loader: ', len(valid_loader))
print('Length of test_loader: ', len(test_loader))

Extracting data/cifar-10-python.tar.gz to data
Images Dimensions:  torch.Size([256, 3, 32, 32])
Labels Dimensions:  torch.Size([256]) 

Images Dimensions:  torch.Size([256, 3, 32, 32])
Labels Dimensions:  torch.Size([256]) 

Images Dimensions:  torch.Size([256, 3, 32, 32])
Labels Dimensions:  torch.Size([256]) 

Length of train_loader:  192
Length of valid_loader:  4
Length of test_loader:  40


In [6]:
torch.manual_seed(RANDOM_SEED)

model = NiN(num_classes=NUM_CLASSES)
model.to(DEVICE)

params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('Total trainable parameters: ', params)

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

Total trainable parameters:  968394


In [7]:
def compute_accuracy(model, data_loader, device):
  correct_pred, num_examples = 0, 0
  for idx, (features, targets) in enumerate(data_loader):
    features = features.to(device)
    targets = targets.to(device)

    logits, probas = model(features)
    _, predicted_labels = torch.max(probas, 1)
    correct_pred += (predicted_labels == targets).sum()
    num_examples += targets.size(0)
  return correct_pred.float()/ num_examples * 100

start_time = time.time()
for epoch in range(EPOCHS):
  model.train()
  for idx, (features, targets) in enumerate(train_loader):
    features = features.to(DEVICE)
    targets = targets.to(DEVICE)

    logits, _ = model(features)

    loss = F.cross_entropy(logits, targets)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # LOGGING
    if idx % 120 == 0:
      print('Batch: %04d/%04d || Epoch: %04d/%04d || Loss: %.3f' % (idx, len(train_loader), epoch+1, EPOCHS, loss.item()))
  
  model.eval()
  with torch.set_grad_enabled(False):
    train_accuracy = compute_accuracy(model, train_loader, DEVICE)
    valid_accuracy = compute_accuracy(model, valid_loader, DEVICE)
    print('Train Accuracy: %.2f' % (train_accuracy))
    print('Valid Accuracy: %.2f' % (valid_accuracy))
  epoch_elapsed_time = (time.time() - start_time) / 60
  print('Epoch Elapsed Time: %.2f min' % (epoch_elapsed_time))
total_elapsed_time = (time.time() - start_time) / 60
print('Total Elapsed Time: %.2f min' % (total_elapsed_time))

Batch: 0000/0192 || Epoch: 0001/0010 || Loss: 2.304
Batch: 0120/0192 || Epoch: 0001/0010 || Loss: 1.134
Train Accuracy: 57.13
Valid Accuracy: 56.00
Epoch Elapsed Time: 0.61 min
Batch: 0000/0192 || Epoch: 0002/0010 || Loss: 0.991
Batch: 0120/0192 || Epoch: 0002/0010 || Loss: 0.840
Train Accuracy: 57.95
Valid Accuracy: 57.00
Epoch Elapsed Time: 1.25 min
Batch: 0000/0192 || Epoch: 0003/0010 || Loss: 0.737
Batch: 0120/0192 || Epoch: 0003/0010 || Loss: 0.713
Train Accuracy: 74.52
Valid Accuracy: 72.70
Epoch Elapsed Time: 1.88 min
Batch: 0000/0192 || Epoch: 0004/0010 || Loss: 0.670
Batch: 0120/0192 || Epoch: 0004/0010 || Loss: 0.679
Train Accuracy: 74.97
Valid Accuracy: 71.50
Epoch Elapsed Time: 2.51 min
Batch: 0000/0192 || Epoch: 0005/0010 || Loss: 0.579
Batch: 0120/0192 || Epoch: 0005/0010 || Loss: 0.622
Train Accuracy: 80.68
Valid Accuracy: 77.00
Epoch Elapsed Time: 3.13 min
Batch: 0000/0192 || Epoch: 0006/0010 || Loss: 0.488
Batch: 0120/0192 || Epoch: 0006/0010 || Loss: 0.552
Train Accur